<a href="https://colab.research.google.com/github/anruki/Quantum-Programming-Applications/blob/main/PauliFeatureMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.datasets import load_iris

## Descarga del dataset
(Solo usamos los datos de Setosa y Versicolor y sus 2 primeras características)

In [11]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

# Cargar el dataset Iris y seleccionar solo las dos primeras características de las clases 'setosa' y 'versicolor'
iris = load_iris()
data = iris.data[iris.target < 2, :2]  # Selecciona solo setosa y versicolor, y solo dos primeras columnas


# Escalar los datos
Entre 0 y pi

In [ ]:
# Escalar los datos en el rango [1, π]
scaler = MinMaxScaler(feature_range=(1, np.pi))
data_scaled = scaler.fit_transform(data)



---



# Ejemplo de `PauliFeatureMap`

In [16]:
from qiskit.circuit.library import PauliFeatureMap
from qiskit import QuantumCircuit

# Crear el PauliFeatureMap con las paulis especificadas y un solo repetidor (reps=1)
prep = PauliFeatureMap(feature_dimension=2, reps=1, paulis=['ZZ'])

# Imprimir el circuito para visualizar su estructura
print(prep.decompose())

     ┌───┐                                          
q_0: ┤ H ├──■────────────────────────────────────■──
     ├───┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐
q_1: ┤ H ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
     └───┘└───┘└──────────────────────────────┘└───┘


# Uso de PauliFeatureMap con 4 qubits y datos escalados

In [19]:
from qiskit import QuantumCircuit

# Crear el circuito cuántico de 4 qubits
qc = QuantumCircuit(4)

# Tomamos los primeros cuatro valores escalados (repetimos x0 y x1)
x0, x1 = data_scaled[0]  # Tomamos la primera muestra
x2, x3 = x0, x1  # Duplicamos para usar en un circuito de 4 qubits

# Aplicar puertas Hadamard a todos los qubits
qc.h(range(4))
qc.barrier()  # Añadir barrera después de las puertas Hadamard

# Aplicar interacciones 'ZZ' entre pares de qubits usando los valores específicos

# Primer par: qubits 0 y 1
qc.cx(0, 1)
qc.p(2 * (np.pi - x0) * (np.pi - x1), 1)
qc.cx(0, 1)
qc.barrier()  # Barrera después del primer par de interacciones

# Segundo par: qubits 2 y 3
qc.cx(2, 3)
qc.p(2 * (np.pi - x2) * (np.pi - x3), 3)
qc.cx(2, 3)
qc.barrier()  # Barrera después del segundo par de interacciones

# Tercer par: qubits 1 y 2 para añadir entrelazamiento adicional
qc.cx(1, 2)
qc.p(2 * (np.pi - x1) * (np.pi - x2), 2)
qc.cx(1, 2)
qc.barrier()  # Barrera después del tercer par de interacciones

# Dibujar el circuito
print(qc)

     ┌───┐ ░                         ░                         ░      »
q_0: ┤ H ├─░───■─────────────────■───░─────────────────────────░──────»
     ├───┤ ░ ┌─┴─┐┌───────────┐┌─┴─┐ ░                         ░      »
q_1: ┤ H ├─░─┤ X ├┤ P(2.4206) ├┤ X ├─░─────────────────────────░───■──»
     ├───┤ ░ └───┘└───────────┘└───┘ ░                         ░ ┌─┴─┐»
q_2: ┤ H ├─░─────────────────────────░───■─────────────────■───░─┤ X ├»
     ├───┤ ░                         ░ ┌─┴─┐┌───────────┐┌─┴─┐ ░ └───┘»
q_3: ┤ H ├─░─────────────────────────░─┤ X ├┤ P(2.4206) ├┤ X ├─░──────»
     └───┘ ░                         ░ └───┘└───────────┘└───┘ ░      »
«                        ░ 
«q_0: ───────────────────░─
«                        ░ 
«q_1: ───────────────■───░─
«     ┌───────────┐┌─┴─┐ ░ 
«q_2: ┤ P(2.4206) ├┤ X ├─░─
«     └───────────┘└───┘ ░ 
«q_3: ───────────────────░─
«                        ░ 


Ahora añadimos el circuito inverso pero con con la segunda fila de iris.